In [1]:
import pandas as pd
import numpy as np

In [2]:
base = pd.read_csv('./input/basestructuredf.csv')
sales = pd.read_csv('./input/cleanedmonthly.csv')
test = pd.read_csv('./input/test.csv')
s_meta = pd.read_csv('./input/shops_meta.csv')
i_meta = pd.read_csv('./input/items_meta.csv')

In [17]:
items = pd.read_csv('./input/items.csv')

In [20]:
items.drop(columns='item_name', inplace=True)

In [3]:
combined = base.merge(sales, 'outer', on=['year', 'date_block_num','month','shop_id', 'item_id'])

In [4]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18648420 entries, 0 to 18648419
Data columns (total 7 columns):
 #   Column                Dtype  
---  ------                -----  
 0   shop_id               int64  
 1   item_id               int64  
 2   date_block_num        int64  
 3   month                 int64  
 4   year                  int64  
 5   item_cnt_month        float64
 6   median_monthly_price  float64
dtypes: float64(2), int64(5)
memory usage: 1.1 GB


In [8]:
def fillnavals_price_item_cnt(df):
    df.median_monthly_price = df.groupby(['shop_id', 'item_id']).median_monthly_price.fillna(method='ffill')
    df.median_monthly_price = df.groupby(['shop_id', 'item_id']).median_monthly_price.fillna(value=0)
    df.fillna(value=0, inplace=True)

In [9]:
fillnavals_price_item_cnt(combined)

In [11]:
combined.sample(10)

,shop_id,item_id,date_block_num,month,year,item_cnt_month,median_monthly_price
10786007,38,6274,22,11,2014,0.0,0.0
184953,2,9056,13,2,2014,0.0,999.0
7073455,24,20600,25,2,2015,0.0,0.0
6066930,21,14671,30,7,2015,0.0,0.0
1649305,5,15740,0,1,2013,0.0,0.0
15148679,50,2453,14,3,2014,0.0,0.0
819117,3,18660,12,1,2014,0.0,99.0
10365640,37,7502,5,6,2013,0.0,0.0
18577063,59,18530,8,9,2013,0.0,0.0
16733854,55,15181,4,5,2013,0.0,0.0


In [12]:
def months_since_last_sale(df):
    df['date_block_with_sale'] = df.apply(lambda o: o.date_block_num + 1 if o.item_cnt_month > 0 else None, axis=1)
    df['date_block_with_sale'] = df.groupby(['shop_id', 'item_id']).date_block_with_sale.fillna(method='ffill')
    df['date_block_with_sale'] = df.groupby(['shop_id', 'item_id']).date_block_with_sale.fillna(value=0)
    lagging_db_with_sale = df.groupby(['shop_id', 'item_id']).date_block_with_sale.shift(1)
    df['months_since_sale'] = df.date_block_num - lagging_db_with_sale
    df.drop(columns='date_block_with_sale', inplace=True)

In [13]:
months_since_last_sale(combined)

In [15]:
combined.loc[(combined.shop_id==59)& (combined.item_id==22167)]

,shop_id,item_id,date_block_num,month,year,item_cnt_month,median_monthly_price,months_since_sale
18648385,59,22167,0,1,2013,0.0,0.0,NaN
18648386,59,22167,1,2,2013,0.0,0.0,1.0
18648387,59,22167,2,3,2013,0.0,0.0,2.0
18648388,59,22167,3,4,2013,0.0,0.0,3.0
18648389,59,22167,4,5,2013,0.0,0.0,4.0
18648390,59,22167,5,6,2013,0.0,0.0,5.0
18648391,59,22167,6,7,2013,0.0,0.0,6.0
18648392,59,22167,7,8,2013,0.0,0.0,7.0
18648393,59,22167,8,9,2013,0.0,0.0,8.0
18648394,59,22167,9,10,2013,1.0,299.0,9.0


In [16]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18648420 entries, 0 to 18648419
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   shop_id               int64  
 1   item_id               int64  
 2   date_block_num        int64  
 3   month                 int64  
 4   year                  int64  
 5   item_cnt_month        float64
 6   median_monthly_price  float64
 7   months_since_sale     float64
dtypes: float64(3), int64(5)
memory usage: 1.9 GB


In [21]:
combined = pd.merge(left=combined, right=items, how='left', left_on='item_id', right_on='item_id')

In [22]:
combined

,shop_id,item_id,date_block_num,month,year,item_cnt_month,median_monthly_price,months_since_sale,item_category_id
0,2,1,0,1,2013,0.0,0.0,NaN,76
1,2,1,1,2,2013,0.0,0.0,1.0,76
2,2,1,2,3,2013,0.0,0.0,2.0,76
3,2,1,3,4,2013,0.0,0.0,3.0,76
4,2,1,4,5,2013,0.0,0.0,4.0,76
...,...,...,...,...,...,...,...,...,...
18648415,59,22167,30,7,2015,0.0,299.0,12.0,49
18648416,59,22167,31,8,2015,0.0,299.0,13.0,49
18648417,59,22167,32,9,2015,0.0,299.0,14.0,49
18648418,59,22167,33,10,2015,0.0,299.0,15.0,49


In [33]:
def add_lagging_values(df, num_months=3, attributes=['item_cnt_month', 'median_monthly_price']):
    for att in attributes:
        for i in range(num_months):
            df[f'{att} - {i+1}'] =  df.groupby(['shop_id', 'item_id'])[att].shift(i+1)
    df.drop(np.where(df.date_block_num.isin(range(num_months)))[0], inplace=True)


In [34]:
add_lagging_values(combined)

In [35]:
combined

,shop_id,item_id,date_block_num,month,year,item_cnt_month,median_monthly_price,months_since_sale,item_category_id,item_cnt_month - 4,median_monthly_price - 4,item_cnt_month - 1,item_cnt_month - 2,item_cnt_month - 3,median_monthly_price - 1,median_monthly_price - 2,median_monthly_price - 3
3,2,1,3,4,2013,0.0,0.0,3.0,76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,1,4,5,2013,0.0,0.0,4.0,76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2,1,5,6,2013,0.0,0.0,5.0,76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2,1,6,7,2013,0.0,0.0,6.0,76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2,1,7,8,2013,0.0,0.0,7.0,76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18648415,59,22167,30,7,2015,0.0,299.0,12.0,49,0.0,299.0,0.0,0.0,0.0,299.0,299.0,299.0
18648416,59,22167,31,8,2015,0.0,299.0,13.0,49,0.0,299.0,0.0,0.0,0.0,299.0,299.0,299.0
18648417,59,22167,32,9,2015,0.0,299.0,14.0,49,0.0,299.0,0.0,0.0,0.0,299.0,299.0,299.0
18648418,59,22167,33,10,2015,0.0,299.0,15.0,49,0.0,299.0,0.0,0.0,0.0,299.0,299.0,299.0
